**재무데이터 활용 최종등급 예측 모델링**



1.   라이브러리 및 데이터 불러오기



In [2]:
!pip install matplotlib

  Using cached matplotlib-3.6.1-cp310-cp310-macosx_10_12_x86_64.whl (7.3 MB)
  Using cached contourpy-1.0.6-cp310-cp310-macosx_10_9_x86_64.whl (240 kB)
  Using cached kiwisolver-1.4.4-cp310-cp310-macosx_10_9_x86_64.whl (65 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     |████████████████████████████████| 3.3 MB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 965 kB 105.9 MB/s eta 0:00:01
You should consider upgrading via the '/Users/jaegyoonoh/.pyenv/versions/3.10.1/bin/python3.10 -m pip install --upgrade pip' command.


In [3]:
!pip install -U scikit-learn

  Using cached scikit_learn-1.1.3-cp310-cp310-macosx_10_9_x86_64.whl (8.7 MB)
  Using cached scipy-1.9.3-cp310-cp310-macosx_10_9_x86_64.whl (34.3 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
You should consider upgrading via the '/Users/jaegyoonoh/.pyenv/versions/3.10.1/bin/python3.10 -m pip install --upgrade pip' command.


In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

In [6]:
#df=pd.read_csv('/content/sampling.csv',sep=",")
df=pd.read_csv('../data/financial_data.csv')


2.   데이터 EDA 및 전처리





In [7]:
df.head()

,BIZ_NO,STT_YR_MO,M11000,M11130,M12000,M12300,M14900,M15000,M16000,M19900-M19800,...,MC2200,MC3100,MD1100,MD1800,MD2300,MD2400,M11150,M22000,M12500,M11500
0,1018100340,20181231,2667731.0,127320.0,32346444.0,1179096.0,35014174.0,422872.0,16176067.0,16598939.0,...,87.841,250.381,0.163,1.000000e+12,44.062,1.000000e+12,NaN,NaN,NaN,NaN
1,1018100340,20191231,2168837.0,94085.0,34151633.0,1209369.0,36320470.0,10553878.0,5240445.0,15794323.0,...,25.500,241.400,0.184,1.000000e+12,59.092,NaN,NaN,NaN,NaN,NaN
2,1018100340,20201231,2272260.0,78612.0,38329680.0,1091153.0,40601940.0,863144.0,15496117.0,16359261.0,...,63.921,275.719,0.154,1.000000e+12,68.714,1.000000e+12,NaN,NaN,NaN,NaN
3,1018100340,20211231,1175030.0,273522.0,37135811.0,993025.0,38310842.0,9494272.0,4901383.0,14395654.0,...,20.495,233.749,0.156,1.000000e+12,34.979,1.000000e+12,3243.0,NaN,NaN,NaN
4,1018100772,20181231,422146.0,17011.0,20370096.0,20352846.0,20792242.0,2684934.0,11404637.0,14089571.0,...,170.151,1250.082,0.055,1.000000e+12,69.711,1.000000e+12,386880.0,524746.0,NaN,NaN


In [8]:
df.shape

(109142, 46)

In [25]:
df1 = df.copy()

In [26]:
df_closed = pd.read_csv('../data/closed_company_summary.csv')

In [27]:
df_closed_filtered = df_closed[['BIZ_NO', 'label']]
df_closed_filtered['LABEL'] = df_closed_filtered['label']
df_closed_filtered = df_closed_filtered.drop(columns=['label'])

/var/folders/w5/9_lbs3zn75j3rp12lcj63_6c0000gn/T/ipykernel_81135/784211479.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_closed_filtered['LABEL'] = df_closed_filtered['label']


In [32]:
df2 = df1.merge(df_closed_filtered, how='left', on='BIZ_NO')

In [45]:
indices = list(df2[df2['LABEL'] != 1].index)

In [46]:
df2.loc[indices, 'LABEL'] = 0

In [47]:
df2.LABEL.value_counts()

0.0    98905
1.0    10237
Name: LABEL, dtype: int64

In [48]:
# 결측치 제거 
df2=df2.dropna()

In [11]:
# 결측 제거 후 확인 
df1.isna().sum()

BIZ_NO           0
STT_YR_MO        0
M11000           0
M11130           0
M12000           0
M12300           0
M14900           0
M15000           0
M16000           0
M19900-M19800    0
M71000           0
M75000           0
M19800           0
M21000           0
M24000           0
M25000           0
M29000           0
M29500           0
M29900           0
MB1500           0
MB2700           0
MB2800           0
MB3100           0
MB3200           0
MB3400           0
MB3500           0
MB3510           0
MB3600           0
MB4200           0
MB5200           0
MB5300           0
MC1600           0
MC1700           0
MC1800           0
MC1900           0
MC2100           0
MC2200           0
MC3100           0
MD1100           0
MD1800           0
MD2300           0
MD2400           0
M11150           0
M22000           0
M12500           0
M11500           0
dtype: int64

In [49]:
df2.head()

,BIZ_NO,STT_YR_MO,M11000,M11130,M12000,M12300,M14900,M15000,M16000,M19900-M19800,...,MC3100,MD1100,MD1800,MD2300,MD2400,M11150,M22000,M12500,M11500,LABEL
20,1018104477,20181231,63617362.0,894220.0,267056570.0,243277730.0,330673932.0,112220541.0,99725297.0,211945838.0,...,558.737,0.113,0.649,43.535,130.062,498897.0,22392335.0,176381.0,53942768.0,0.0
21,1018104477,20191231,60921607.0,913618.0,264034556.0,238156706.0,324956163.0,125448350.0,77764751.0,203213101.0,...,294.400,0.210,1.772,76.354,265.310,241713.0,49058836.0,125250.0,23951998.0,0.0
22,1018104477,20201231,43451141.0,888892.0,233044178.0,206314843.0,276495319.0,94889526.0,58084652.0,152974178.0,...,377.699,0.135,2.204,44.939,184.692,358110.0,27590642.0,69734.0,12803265.0,0.0
23,1018104477,20211231,56571216.0,883688.0,230454713.0,200553518.0,287025929.0,95917746.0,45122291.0,141040037.0,...,310.874,0.161,5.611,51.190,255.887,58707.0,27569619.0,72385.0,3367175.0,0.0
52,1018119252,20181231,6918597.0,160635.0,7720331.0,5805367.0,14638928.0,3105215.0,3500000.0,6605215.0,...,77.390,0.600,1.584,34.826,44.765,10055.0,2985546.0,31948.0,5420362.0,0.0





3.   모델링



In [50]:
# features/target, train/test dataset 분리
# 사업자번호의 경우 최종등급의 영향이 크지 않을것으로 예측되어 제외
X = df2.drop(columns=['LABEL'])
y = df2[['LABEL']]

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할| 

In [51]:
## 데이터 스케일링
# 데이터 스케일링 작업을 통해, 모든 특성의 범위(또는 분포)를 같게 만들어줘야합니다.

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [61]:
## 랜덤포레스트 모델 생성 및 평가 
model_rf = RandomForestClassifier(n_estimators=20, max_depth=5,random_state=0)
model_rf.fit(train_x,train_y)

# SCORE 산출
predict1 = model_rf.predict(test_x)
print(f'ACC: {accuracy_score(test_y, predict1)}')
print(f'AUC: {roc_auc_score(test_y, predict1)}')
# 중요도
print(f'변수 중요도: \n{model_rf.feature_importances_}')

/var/folders/w5/9_lbs3zn75j3rp12lcj63_6c0000gn/T/ipykernel_81135/2210117533.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rf.fit(train_x,train_y)


ACC: 0.9612291194060631
AUC: 0.5870116072346011
변수 중요도: 
[8.47707810e-03 1.02057136e-01 7.24448251e-02 5.10194903e-03
 5.59113677e-02 3.86397868e-02 1.16690890e-01 5.73371602e-02
 1.64693631e-02 7.72568699e-02 2.33760175e-02 4.04281677e-02
 4.27869806e-02 8.17745362e-02 5.79781996e-02 3.49107677e-02
 1.84262524e-02 1.29561394e-02 3.48439556e-02 0.00000000e+00
 0.00000000e+00 3.16932464e-04 0.00000000e+00 0.00000000e+00
 3.51508715e-04 3.12092319e-04 1.17374310e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.89227331e-04 0.00000000e+00
 0.00000000e+00 8.09288230e-05 2.58284224e-02 1.97108816e-03
 7.02701466e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 5.93370498e-05 6.42661682e-03 3.21665663e-02
 1.07585903e-02 1.11311141e-02]


In [60]:
## 로지스틱 모델 생성 및 평가 

model_lg = LogisticRegression()
model_lg.fit(train_x, train_y)

# SCORE 산출
predict2 = model_lg.predict(test_x)
print(f'ACC: {accuracy_score(test_y, predict2)}')
print(f'AUC: {roc_auc_score(test_y, predict2)}')

/Users/jaegyoonoh/.pyenv/versions/3.10.1/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ACC: 0.952567539698907
AUC: 0.5077796777426866


/Users/jaegyoonoh/.pyenv/versions/3.10.1/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [55]:
# 새로운 데이터 예측하기 

jack=np.array([504,11,4,8888,0,2,70,6])
me=np.array([504,11,4,9999,100,4,20,3])
i=np.array([504,11,4,7777,0,2,70,2])

sample_passengers = np.array([jack, me,i])

# 새로운 데이터도 스케일링 해줘야함.
sample_passengers = scaler.transform(sample_passengers)

/Users/jaegyoonoh/.pyenv/versions/3.10.1/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: X has 8 features, but StandardScaler is expecting 46 features as input.

In [54]:
# 새로운 데이터에 대해 최종등급 예측

print(model_rf.predict(sample_passengers))
print(model_lg.predict(sample_passengers))

NameError: name 'sample_passengers' is not defined



4.   제출파일 생성



In [ ]:
predict1 = model.predict(test_x)

In [ ]:
#제출용 파일 infile
submission=pd.read_csv('/content/submission.csv')
submission

In [ ]:
predict1

array([0., 5., 0., ..., 1., 0., 2.])

In [ ]:
submission['최종등급']=predict1
submission

In [ ]:
submission.to_csv('결과물제출.csv')